# Identifying Customer Geographies

As an analyst for ProWidget Systems, a UK-based B2B (business-to-business) retailer, you’ve been asked to report on spending volumes for London-based customers versus those based in the rest of the United Kingdom. The longterm goal is to better understand our customer base. The board has supplied a high-level data extract containing all customers’ addresses and their total spending to date. They want to know:

1. Which UK cities are currently underserved
2. Whether their customers are primarily London based


In [1]:
! conda env create -f environment.yml
! conda list | sort



CondaValueError: prefix already exists: /Users/chris/opt/anaconda3/envs/data_science

#
# Name                    Version                   Build  Channel
# packages in environment at /Users/chris/opt/anaconda3/envs/data_science:
adbc-driver-manager       1.4.0                    pypi_0    pypi
adbc-driver-sqlite        1.4.0                    pypi_0    pypi
altair                    5.5.0                    pypi_0    pypi
annotated-types           0.7.0                    pypi_0    pypi
appnope                   0.1.3           py313hecd8cb5_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
attrs                     25.1.0                   pypi_0    pypi
babel                     2.17.0                   pypi_0    pypi
blas                      2.131                  openblas    conda-forge
blas-devel                3.9.0           31_hbf4f893_openblas    conda-forge
bzip2                     1.0.8                h6c40b1e_6  
ca-certificates           2025.2.25       

# Environment Setup

In [2]:
import polars as pl
import plotly.express as px

In [67]:
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(100)
pl.Config.set_fmt_str_lengths(500)

polars.config.Config

In [118]:
prep_addresses_df = pl.read_csv("./data/addresses.csv")
prep_cities_df = pl.read_csv("./data/cities.csv", has_header=False).rename({"column_1": "city"})

# Business Understanding
We're exploring a business intelligence project aimed on understanding our customer base's geographical distribution. Our board has specifically requested insights into whether the business is overly London-centric and which UK cities might represent untapped market potential. This analysis will serve as a foundation for strategic market expansion and resource allocation decisions.

Our primary objectives are to quantify the London versus non-London customer base distribution and identify underserved markets across the UK. Success in this project will be measured by our ability to provide actionable geographic insights, including a clear understanding of current market penetration across different cities and specific recommendations for market expansion opportunities.

# Data Understanding
We have access to two primary data sources: a customer database containing address information and spending data (`addresses.csv`), and a reference list of UK cities (`cities.csv`). The customer data presents a particular challenge as it contains free-text address fields, which will require careful processing to extract standardized location information.

Our initial data exploration will focus on understanding the various formats and patterns in the address data, assessing data completeness, and examining the distribution of customer spending. We'll need to pay particular attention to how cities are represented in the free text and identify any systematic patterns or variations in how addresses are recorded. The quality of our city reference data will also be crucial for accurate geographical classification.


## addresses

In [4]:
display(prep_addresses_df.head(10))
display(prep_addresses_df.describe())


company_id,address,total_spend
i64,str,i64
1,"""APARTMENT 2, 52 BEDFORD ROAD, …",5700
2,"""107 SHERINGHAM AVENUE, LONDON,…",4700
3,"""43 SUNNINGDALE, YATE, BRISTOL,…",5900
4,"""HAWESWATER HOUSE, LINGLEY MERE…",7200
5,"""AMBERFIELD BARN HOUSE AMBER LA…",4600
6,"""FIRST FLOOR CEF BUILDING, BROO…",4600
7,"""37 BEVERLEY ROAD, DRIFFIELD, E…",7300
8,"""B82 ALBION RIVERSIDE BUILDING,…",6100
9,"""Flat 27, Domelton House, Iron …",4200


statistic,company_id,address,total_spend
str,f64,str,f64
"""count""",100000.0,"""99032""",100000.0
"""null_count""",0.0,"""968""",0.0
"""mean""",50000.5,null,4951.662
"""std""",28867.657797,null,1500.983866
"""min""",1.0,"""""Broadridge"" 55 Jacksons Lane,…",0.0
"""25%""",25001.0,null,3900.0
"""50%""",50001.0,null,5000.0
"""75%""",75000.0,null,6000.0
"""max""",100000.0,"""İSTIKLAL MAH., BARIŞ MANÇO CAD…",11700.0


In [6]:
"""
1% of customers have no address
"""

null_counts = prep_addresses_df.null_count()
total_counts = prep_addresses_df.shape[0]
null_percentages = (null_counts / total_counts) * 100
null_data = {
    "column": prep_addresses_df.columns,
    "null_counts": null_counts,
    "null_percentages": null_percentages
}
null_data

{'column': ['company_id', 'address', 'total_spend'],
 'null_counts': shape: (1, 3)
 ┌────────────┬─────────┬─────────────┐
 │ company_id ┆ address ┆ total_spend │
 │ ---        ┆ ---     ┆ ---         │
 │ u32        ┆ u32     ┆ u32         │
 ╞════════════╪═════════╪═════════════╡
 │ 0          ┆ 968     ┆ 0           │
 └────────────┴─────────┴─────────────┘,
 'null_percentages': shape: (1, 3)
 ┌────────────┬─────────┬─────────────┐
 │ company_id ┆ address ┆ total_spend │
 │ ---        ┆ ---     ┆ ---         │
 │ f64        ┆ f64     ┆ f64         │
 ╞════════════╪═════════╪═════════════╡
 │ 0.0        ┆ 0.968   ┆ 0.0         │
 └────────────┴─────────┴─────────────┘}

In [127]:
for address in prep_addresses_df["address"][:10]:
    print(address, end="\n\n")

APARTMENT 2,
52 BEDFORD ROAD,
LONDON,
ENGLAND,
SW4 7HJ

107 SHERINGHAM AVENUE,
LONDON,
N14 4UJ

43 SUNNINGDALE,
YATE,
BRISTOL,
ENGLAND,
BS37 4HZ

HAWESWATER HOUSE,
LINGLEY MERE BUSINESS PARK,
LINGLEY GREEN AVENUE,
GREAT SANKEY, WARRINGTON,
WA5 3LP

AMBERFIELD BARN HOUSE AMBER LANE,
CHART SUTTON,
MAIDSTONE,
ENGLAND,
ME17 3SF

FIRST FLOOR CEF BUILDING,
BROOMHILL WAY,
TORQUAY,
DEVON,
UNITED KINGDOM,
TQ2 7QN

37 BEVERLEY ROAD,
DRIFFIELD,
ENGLAND,
YO25 6RZ

B82 ALBION RIVERSIDE BUILDING,
8 HESTER ROAD,
LONDON,
UNITED KINGDOM,
SW11 4AP

Flat 27, Domelton House,
Iron Mill Road,
London,
England,
Sw18 2Af

34 BROOKSIDE ESTATE,
CHALGROVE,
OXFORD,
UNITED KINGDOM,
OX44 7SQ



## cities

In [119]:
"""
will need to clean up city names and remove trailing asterisks
"""
display(prep_cities_df.head(10))
display(prep_cities_df.describe())

city
str
"""England"""
null
"""Bath"""
"""Birmingham*"""
"""Bradford*"""
"""Brighton & Hove"""
"""Bristol*"""
"""Cambridge"""
"""Canterbury*"""


statistic,city
str,str
"""count""","""80"""
"""null_count""","""7"""
"""mean""",null
"""std""",null
"""min""","""Aberdeen*"""
"""25%""",null
"""50%""",null
"""75%""",null
"""max""","""York*"""


check nulls

In [120]:
"""
8% of cities are null
"""

null_counts = prep_cities_df.null_count()
total_counts = prep_cities_df.shape[0]
null_percentages = (null_counts / total_counts) * 100
null_data = {
    "column": prep_cities_df.columns,
    "null_counts": null_counts,
    "null_percentages": null_percentages
}
null_data

{'column': ['city'],
 'null_counts': shape: (1, 1)
 ┌──────┐
 │ city │
 │ ---  │
 │ u32  │
 ╞══════╡
 │ 7    │
 └──────┘,
 'null_percentages': shape: (1, 1)
 ┌──────────┐
 │ city     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 8.045977 │
 └──────────┘}

In [121]:
"""
- country headings were included as rows in the data, so the values England, Scotland, Wales, and Northern Ireland need to be removed.
- the trailing asterisk * character also needs to be trimmed
- city names should be uppercased to match our address data

"""
# from pprint import pprint
print(prep_cities_df.to_dict())

{'city': shape: (87,)
Series: 'city' [str]
[
	"England"
	null
	"Bath"
	"Birmingham*"
	"Bradford*"
	"Brighton & Hove"
	"Bristol*"
	"Cambridge"
	"Canterbury*"
	"Carlisle"
	"Chelmsford"
	"Chester*"
	"Chichester"
	"Colchester"
	"Coventry*"
	"Derby"
	"Doncaster"
	"Durham"
	"Ely"
	"Exeter*"
	"Gloucester"
	"Hereford"
	"Kingston-upon-Hull*"
	"Lancaster"
	"Leeds*"
	"Leicester*"
	"Lichfield"
	"Lincoln"
	"Liverpool*"
	"London*"
	"Manchester*"
	"Milton Keynes"
	"Newcastle-upon-Tyne*"
	"Norwich*"
	"Nottingham*"
	"Oxford*"
	"Peterborough"
	"Plymouth*"
	"Portsmouth*"
	"Preston"
	"Ripon"
	"Salford"
	"Salisbury"
	"Sheffield*"
	"Southampton*"
	"Southend-on-Sea"
	"St Albans"
	"Stoke on Trent*"
	"Sunderland"
	"Truro"
	"Wakefield"
	"Wells"
	"Westminster*"
	"Winchester"
	"Wolverhampton"
	"Worcester"
	"York*"
	null
	"Northern Ireland"
	null
	"Armagh*"
	"Bangor"
	"Belfast*"
	"Lisburn"
	"Londonderry"
	"Newry"
	null
	"Scotland"
	null
	"Aberdeen*"
	"Dundee*"
	"Dunfermline"
	"Edinburgh*"
	"Glasgow*"
	"Inverness"


In [122]:
"""
there are duplicates in the cities data
    - several nulls
    - multiple "Bangor"
"""
prep_cities_df.is_duplicated().sum()

prep_cities_df.group_by("city").agg(pl.len().alias("count")).sort("count", descending=True).filter(pl.col("count") > 1)

city,count
str,u32
null,7
"""Bangor""",2


# Data Preparation
The data preparation phase will center around transforming our free-text address data into structured geographical insights. This will involve developing robust text processing routines to standardize addresses and reliably extract city information. We'll need to handle various edge cases, such as different spellings of city names, multiple locations in a single address, and potential ambiguities in location references.

We'll create several derived features to support our analysis, including a clear London/non-London classification and regional groupings where appropriate. The integration of our processed address data with the reference city list will be crucial for ensuring consistent geographical classification. This stage will also involve creating aggregate metrics at various geographical levels to support our analysis.

In [303]:
addresses_df = prep_addresses_df.drop_nulls(subset=["address"])
cities_df = prep_cities_df.drop_nulls(subset=["city"])

## addresses

In [304]:
display(addresses_df.describe())

statistic,company_id,address,total_spend
str,f64,str,f64
"""count""",99032.0,"""99032""",99032.0
"""null_count""",0.0,"""0""",0.0
"""mean""",50004.016964,null,4951.673197
"""std""",28867.36374,null,1500.642398
"""min""",1.0,"""""Broadridge"" 55 Jacksons Lane, Hazel Grove, Stockport, Cheshire, Sk7 5Jp""",0.0
"""25%""",25004.0,null,3900.0
"""50%""",49999.0,null,5000.0
"""75%""",75008.0,null,6000.0
"""max""",100000.0,"""İSTIKLAL MAH., BARIŞ MANÇO CAD. NO:2, ESENYURT/İSTANBUL""",11700.0


In [305]:
addresses_df = addresses_df.with_columns([
    pl.col("address").str.to_uppercase().alias("address_clean")
])

for address in addresses_df["address_clean"][:10]:
    print(address, end="\n\n")

APARTMENT 2,
52 BEDFORD ROAD,
LONDON,
ENGLAND,
SW4 7HJ

107 SHERINGHAM AVENUE,
LONDON,
N14 4UJ

43 SUNNINGDALE,
YATE,
BRISTOL,
ENGLAND,
BS37 4HZ

HAWESWATER HOUSE,
LINGLEY MERE BUSINESS PARK,
LINGLEY GREEN AVENUE,
GREAT SANKEY, WARRINGTON,
WA5 3LP

AMBERFIELD BARN HOUSE AMBER LANE,
CHART SUTTON,
MAIDSTONE,
ENGLAND,
ME17 3SF

FIRST FLOOR CEF BUILDING,
BROOMHILL WAY,
TORQUAY,
DEVON,
UNITED KINGDOM,
TQ2 7QN

37 BEVERLEY ROAD,
DRIFFIELD,
ENGLAND,
YO25 6RZ

B82 ALBION RIVERSIDE BUILDING,
8 HESTER ROAD,
LONDON,
UNITED KINGDOM,
SW11 4AP

FLAT 27, DOMELTON HOUSE,
IRON MILL ROAD,
LONDON,
ENGLAND,
SW18 2AF

34 BROOKSIDE ESTATE,
CHALGROVE,
OXFORD,
UNITED KINGDOM,
OX44 7SQ



In [306]:
# 21768 customers with "London" in their address
addresses_df.select("address_clean").filter(pl.col("address_clean").str.contains("LONDON")).count()

# 20831 customers in the city of London
addresses_df.select("address_clean").filter(pl.col("address_clean").str.contains("LONDON,")).count()

address_clean
u32
20831


In [ ]:
# split address into lines/parts
addresses_df = addresses_df.with_columns([
    pl.col("address_clean").str.split(by=",\n").alias("address_split")
])

# count the number of lines in the address
addresses_df = addresses_df.with_columns([
    pl.col("address_split").list.len().alias("address_lines")
])
addresses_df.head(10)


"""
Some addresses consist of only one or two lines, and some have as many as six.
"""
_ = addresses_df.group_by("address_lines").agg(
    pl.len().alias("count")
).sort("address_lines")

customer_spending_fig = px.bar(_, x="address_lines", y="count", title="Address Line Frequency", text="count")
customer_spending_fig.update_layout(
    height=600,  # Set a taller height to accommodate all elements
    autosize=True,  # Enable autosize for responsive behavior
    yaxis=dict(
        type='log',  # Apply log scale to y-axis
        title='Count (log scale)'  # Update y-axis title to indicate log scale
    ),
    annotations=[
        dict(
            text="Distribution of address line counts in customer data — some addresses consist of only one or two lines, and some have as many as six.",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=-0.2,
            font=dict(size=12)
        )
    ],
    margin=dict(b=100)  # Add bottom margin to ensure annotation is visible
)
customer_spending_fig.update_traces(textposition='outside')
customer_spending_fig.show()

In [139]:
for num_lines in range(1, 7):
    display(f"Number of lines: {num_lines}")
    display(addresses_df.select(["address_clean", "address_lines"])\
            .filter(pl.col("address_lines") == num_lines)\
                .select("address_clean").head(1)
            )


'Number of lines: 1'

address_clean
str
"""FALKIRK"""


'Number of lines: 2'

address_clean
str
"""85 GLASGOW ROAD, DUMBARTON"""


'Number of lines: 3'

...

address_clean
str
"""37 BEVERLEY ROAD, DRIFFIELD, ENGLAND, YO25 6RZ"""


'Number of lines: 5'

address_clean
str
"""APARTMENT 2, 52 BEDFORD ROAD, LONDON, ENGLAND, SW4 7HJ"""


'Number of lines: 6'

address_clean
str
"""FIRST FLOOR CEF BUILDING, BROOMHILL WAY, TORQUAY, DEVON, UNITED KINGDOM, TQ2 7QN"""


## cities

In [299]:
display(cities_df.describe())

statistic,city
str,str
"""count""","""76"""
"""null_count""","""0"""
"""mean""",null
"""std""",null
"""min""","""ABERDEEN"""
"""25%""",null
"""50%""",null
"""75%""",null
"""max""","""YORK"""


In [308]:
countries_to_remove = ["England", "Scotland", "Wales", "Northern Ireland"]
cities_df = cities_df.filter(~pl.col("city").is_in(countries_to_remove))
cities_df = cities_df.with_columns([
    pl.col("city")\
        .str.replace("*", "", literal=True)\
        .str.to_uppercase()\
        .sort()\
        .alias("city")
])

cities_df


city
str
"""ABERDEEN"""
"""ARMAGH"""
"""BANGOR"""
"""BANGOR"""
"""BATH"""
"""BELFAST"""
"""BIRMINGHAM"""
"""BRADFORD"""
"""BRIGHTON & HOVE"""


## customers

In [309]:
# Create a new column 'city' in addresses_df using a simpler approach
# Use pattern matching to extract city from address_clean
# addresses_df =

# Create a set of cities for faster lookup
cities_set = set(cities_df["city"].to_list())

# map the city to the address
customers_df = addresses_df.with_columns([
    pl.col("address_clean").map_elements(
        lambda addr: next(
            (city for city in cities_set if f"\n{city}," in addr),
            "OTHER"
        ),
        return_dtype=pl.Utf8
    ).alias("city")
])

display(customers_df.select(["company_id", "address_clean", "city"]).head(10))

company_id,address_clean,city
i64,str,str
1,"""APARTMENT 2, 52 BEDFORD ROAD, LONDON, ENGLAND, SW4 7HJ""","""LONDON"""
2,"""107 SHERINGHAM AVENUE, LONDON, N14 4UJ""","""LONDON"""
3,"""43 SUNNINGDALE, YATE, BRISTOL, ENGLAND, BS37 4HZ""","""BRISTOL"""
4,"""HAWESWATER HOUSE, LINGLEY MERE BUSINESS PARK, LINGLEY GREEN AVENUE, GREAT SANKEY, WARRINGTON, WA5 3LP""","""OTHER"""
5,"""AMBERFIELD BARN HOUSE AMBER LANE, CHART SUTTON, MAIDSTONE, ENGLAND, ME17 3SF""","""OTHER"""
6,"""FIRST FLOOR CEF BUILDING, BROOMHILL WAY, TORQUAY, DEVON, UNITED KINGDOM, TQ2 7QN""","""OTHER"""
7,"""37 BEVERLEY ROAD, DRIFFIELD, ENGLAND, YO25 6RZ""","""OTHER"""
8,"""B82 ALBION RIVERSIDE BUILDING, 8 HESTER ROAD, LONDON, UNITED KINGDOM, SW11 4AP""","""LONDON"""
9,"""FLAT 27, DOMELTON HOUSE, IRON MILL ROAD, LONDON, ENGLAND, SW18 2AF""","""LONDON"""


In [365]:
"""
over 50% of customers are not in London or major cities
"""

total_customers_df = customers_df.select("city")\
    .group_by("city")\
    .agg(pl.len().alias("count"))\
    .sort("count", descending=True)\

total_count = _["count"].sum()
total_customers_df = total_customers_df.with_columns((pl.col("count") / total_count * 100).round(2).alias("percentage"))

total_customers_df_count_fig = px.bar(_.top_k(10, by="count"), x="city", y="count", title="Top 10 Customer Cities (by count)")
total_customers_df_count_fig.update_layout(
    height=600,
    autosize=True,
    yaxis=dict(
        title='Count'
    ),
    xaxis=dict(
    )
).show()

total_customers_df_pct_fig = px.bar(_.top_k(10, by="count"), x="city", y="percentage", title="Top 10 Customer Cities (by percentage)")
total_customers_df_pct_fig.update_layout(
    height=600,  # Increased height to make the viewing area taller
    autosize=True,
    yaxis=dict(
        title='Percentage'
    ),
    xaxis=dict(
        title=None,
        tickangle=45
    ),
    annotations=[
        dict(
            text="Over 50% of customers are not in London or major cities.",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=1,
            y=-0.2,  # Further adjusted y position to move below the x-axis title
            font=dict(size=12)
        )
    ],
)
total_customers_df_pct_fig.update_traces(textposition='outside').show()


In [311]:
"""
"OTHER" locations are not London or major cities
"""
for address in customers_df.filter(pl.col("city") == "OTHER").select("address_clean").to_series().head(10):
    print(address, end="\n\n")


HAWESWATER HOUSE,
LINGLEY MERE BUSINESS PARK,
LINGLEY GREEN AVENUE,
GREAT SANKEY, WARRINGTON,
WA5 3LP

AMBERFIELD BARN HOUSE AMBER LANE,
CHART SUTTON,
MAIDSTONE,
ENGLAND,
ME17 3SF

FIRST FLOOR CEF BUILDING,
BROOMHILL WAY,
TORQUAY,
DEVON,
UNITED KINGDOM,
TQ2 7QN

37 BEVERLEY ROAD,
DRIFFIELD,
ENGLAND,
YO25 6RZ

12 BORELLI YARD,
FARNHAM,
SURREY,
ENGLAND,
GU9 7NU

ORCHARD FARM BROOKHOUSE GREEN,
SMALLWOOD,
SANDBACH,
CHESHIRE,
ENGLAND,
CW11 2XE

4 1ST FLOOR, 4 EARLS COURT,
EARLS GATE PARK,
GRANGEMOUTH,
SCOTLAND,
FK3 8ZE

15 ILFRACOMBE GARDENS,
ROMFORD,
ENGLAND,
RM6 4RL

11 RICHMOND ROAD,
CHORLEY,
ENGLAND,
PR6 0NS

103 ST. STEPHENS WALK,
ASHFORD,
KENT,
UNITED KINGDOM,
TN23 5BE



In [312]:
"""
cities in cities_df but not in addresses_df:
KINGSTON-UPON-HULL
    - KINGSTON-UPON-HULL is a commonly shorten to "Hull"
WESTMINSTER
"""
set(cities_df["city"]) - set(customers_df["city"])

{'KINGSTON-UPON-HULL', 'WESTMINSTER'}

In [316]:
customers_df = customers_df.with_columns([
    pl.when(pl.col("address_clean").str.contains("\nHULL,"))
      .then(pl.lit("HULL"))
      .otherwise(pl.col("city"))
      .alias("city")
])

set(cities_df["city"]) - set(customers_df["city"])

customers_df.filter(pl.col("address_clean").str.contains("\nHULL,"))

company_id,address,total_spend,address_clean,address_split,address_lines,city
i64,str,i64,str,list[str],u32,str
456,"""349 BEVERLEY ROAD, ANLABY, HULL, ENGLAND, HU10 7BQ""",3800,"""349 BEVERLEY ROAD, ANLABY, HULL, ENGLAND, HU10 7BQ""","[""349 BEVERLEY ROAD"", ""ANLABY"", … ""HU10 7BQ""]",5,"""HULL"""
1140,"""90 CALVERT LANE, HULL, ENGLAND, HU4 6BJ""",6000,"""90 CALVERT LANE, HULL, ENGLAND, HU4 6BJ""","[""90 CALVERT LANE"", ""HULL"", … ""HU4 6BJ""]",4,"""HULL"""
1823,"""ACER GLADE BLACK TUP LANE, ARNOLD, HULL, ENGLAND, HU11 5JA""",4600,"""ACER GLADE BLACK TUP LANE, ARNOLD, HULL, ENGLAND, HU11 5JA""","[""ACER GLADE BLACK TUP LANE"", ""ARNOLD"", … ""HU11 5JA""]",5,"""HULL"""
1832,"""10 GORDON STREET, HULL, ENGLAND, HU3 3HN""",5300,"""10 GORDON STREET, HULL, ENGLAND, HU3 3HN""","[""10 GORDON STREET"", ""HULL"", … ""HU3 3HN""]",4,"""HULL"""
1922,"""70 WRIGHT STREET, HULL, ENGLAND, HU2 8JD""",6100,"""70 WRIGHT STREET, HULL, ENGLAND, HU2 8JD""","[""70 WRIGHT STREET"", ""HULL"", … ""HU2 8JD""]",4,"""HULL"""
2138,"""32 KING EDWARD STREET, HULL, ENGLAND, HU1 3SS""",7300,"""32 KING EDWARD STREET, HULL, ENGLAND, HU1 3SS""","[""32 KING EDWARD STREET"", ""HULL"", … ""HU1 3SS""]",4,"""HULL"""
2345,"""Riverside House, 11-12 Nelson Street, Hull, England, Hu1 1Xe""",1700,"""RIVERSIDE HOUSE, 11-12 NELSON STREET, HULL, ENGLAND, HU1 1XE""","[""RIVERSIDE HOUSE"", ""11-12 NELSON STREET"", … ""HU1 1XE""]",5,"""HULL"""
2349,"""5 MARITIME HOUSE MARITIME BUSINESS PARK, LIVINGSTON ROAD, HESSLE, HULL, ENGLAND, HU13 0EG""",6000,"""5 MARITIME HOUSE MARITIME BUSINESS PARK, LIVINGSTON ROAD, HESSLE, HULL, ENGLAND, HU13 0EG""","[""5 MARITIME HOUSE MARITIME BUSINESS PARK"", ""LIVINGSTON ROAD"", … ""HU13 0EG""]",6,"""HULL"""
2892,"""368 STAVELEY ROAD, HULL, ENGLAND, HU9 4BY""",1800,"""368 STAVELEY ROAD, HULL, ENGLAND, HU9 4BY""","[""368 STAVELEY ROAD"", ""HULL"", … ""HU9 4BY""]",4,"""HULL"""


<!-- # Modeling -->

# Evaluation


In [362]:
customer_spending_by_city = customers_df.group_by("city")\
    .agg((pl.col("total_spend")/1_000_000)\
        .sum().round(2)\
        .alias("total_spending"))\
    .top_k(20, by="total_spending")\
    .sort("total_spending", descending=False)

customer_spending_fig = px.bar(customer_spending_by_city, y="city", x="total_spending", title="Total Customer Spending by City", text="total_spending")

customer_spending_fig.update_layout(
    height=600,
    autosize=True,
    yaxis=dict(
        title='Total Spending'
    ),
    xaxis=dict(
        tickformat=",.2f",
        tickprefix="£",
        ticksuffix="M"
    ),
    annotations=[
        dict(
            text="Over 50% of customers are not in London or major cities.",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=1,
            y=-0.2,  # Further adjusted y position to move below the x-axis title
            font=dict(size=12)
        )
    ],
)

customer_spending_fig.update_traces(textposition='outside')

customer_spending_fig.show()


## observations
- Most of the customers don't reside in the 76 cities from the list
- The cities with the most customers and the most spending are:
    - London
    - Manchester
    - Birmingham
- The top 10 cities in the customer count chart generally corresponds with the top cities in the customer spending chart
- London customer spending is almost as much as the combined spending for larger cities — meaning customer base is focused around London


In [366]:
total_customers_df_count_fig.show()
customer_spending_fig.show()


Our final inquiry is to compare London against the rest of the United Kingdom. We can take this to mean either “all of the United Kingdom that isn’t London” or “all major cities except London.” 

In [ ]:
tot_spend = customers_df.select(pl.col("total_spend").sum()).item()/1_000_000
print(f"Total spend for all customers: £{tot_spend:,.2f}M")

total_spend_london = customers_df.filter(pl.col("city") == "LONDON").select(pl.col("total_spend").sum()).item()/1_000_000
print(f"Total spend for London customers: £{total_spend_london:,.2f}M")

total_spend_non_london = tot_spend - total_spend_london
print(f"Total spend for non-London customers: £{total_spend_non_london:,.2f}M")

total_spend_non_london_non_other = customers_df.filter(~pl.col("city").is_in(["LONDON", "OTHER"])).select(pl.col("total_spend").sum()).item()/1_000_000
print(f"Total spend for non-London and non-Other customers: £{total_spend_non_london_non_other:,.2f}M")

Total spend for all customers: £490.37M
Total spend for London customers: £103.09M
Total spend for non-London customers: £387.28M
Total spend for non-London and non-Other customers: £119.42M


# Technical Implementation Notes


In [13]:
import polars

<b>Caveats of our analysis</b>:

1. Based on the method we've chosen, not all addresses are in a format that allows for the correct city extraction.
1. We have used the government’s list of cities, which excludes larger towns that stakeholders may be interested in. It is possible to make our city-level figures more accurate and more granular.

<b>Potential future analysis</b>:
1. Augment our data with population statistics about each city or town. This could help us understand whether there are cities where customers are spending less than expected based on their city’s population and are, therefore, being underserved.